In [439]:
import pandas as pd
import numpy as np

In [440]:
train = pd.read_csv('option_train.csv')
test = pd.read_csv('option_test_wolabel.csv')

In [441]:
train = train.replace({'BS': {'Under':0, 'Over':1}})

In [442]:
train.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420,0.341270,0.03013,0
1,0.125000,427.015526,465,0.166667,0.03126,1
2,20.691244,427.762336,415,0.265873,0.03116,0
3,1.035002,451.711658,460,0.063492,0.02972,1
4,39.553020,446.718974,410,0.166667,0.02962,0


In [443]:
train.shape

(1680, 6)

In [444]:
train.describe()

,Value,S,K,tau,r,BS
count,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000
mean,15.064384,440.901494,438.247024,0.201930,0.030235,0.436905
std,14.035795,7.527497,23.396936,0.099712,0.000557,0.496151
min,0.125000,425.472331,375.000000,0.003968,0.029510,0.000000
25%,2.255001,433.863864,420.000000,0.119048,0.029820,0.000000
50%,11.190967,442.579723,440.000000,0.202381,0.030130,0.000000
75%,25.744375,447.320414,455.000000,0.285714,0.030540,1.000000
max,60.149367,455.880619,500.000000,0.392857,0.031880,1.000000


# Creating Variables

In [445]:
new = train.copy()
new = new.drop(['BS'], axis=1)

In [446]:
col = list(new.columns)
col_r = col[::-1]

In [447]:
for i in new.columns:
    for j in col_r:
        if i==j:
            col_r.remove(i)
        else:
            new[i+'_times_'+j] = new[i]*new[j]
            new[i+'_by_'+j] = new[i]/new[j]

In [448]:
new = new.replace([np.inf, -np.inf], np.nan)

In [449]:
new.head()

,Value,S,K,tau,r,Value_times_r,Value_by_r,Value_times_tau,Value_by_tau,Value_times_K,...,S_times_tau,S_by_tau,S_times_K,S_by_K,K_times_r,K_by_r,K_times_tau,K_by_tau,tau_times_r,tau_by_r
0,21.670404,431.623898,420,0.341270,0.03013,0.652929,719.230152,7.395455,63.499325,9101.569877,...,147.300219,1264.758401,181282.037286,1.027676,12.6546,13939.595088,143.333333,1230.697675,0.010282,11.326580
1,0.125000,427.015526,465,0.166667,0.03126,0.003908,3.998720,0.020833,0.750000,58.125001,...,71.169254,2562.093150,198562.219543,0.918313,14.5359,14875.239923,77.500000,2789.999994,0.005210,5.331627
2,20.691244,427.762336,415,0.265873,0.03116,0.644739,664.032230,5.501244,77.823784,8586.866380,...,113.730463,1608.897145,177521.369606,1.030753,12.9314,13318.356868,110.337302,1560.895522,0.008285,8.532510
3,1.035002,451.711658,460,0.063492,0.02972,0.030760,34.825101,0.065714,16.301282,476.100926,...,28.680105,7114.458665,207787.362588,0.981982,13.6712,15477.792732,29.206349,7245.000056,0.001887,2.136341
4,39.553020,446.718974,410,0.166667,0.02962,1.171560,1335.348412,6.592170,237.318119,16216.738180,...,74.453163,2680.313841,183154.779504,1.089558,12.1442,13841.998650,68.333333,2459.999995,0.004937,5.626829


# Model

## Feature Selection

In [ ]:
# may need to try out other methods for feature selection

In [494]:
train_final = new.copy()

In [495]:
X = train_final
y = train['BS']

In [496]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1700, penalty='none')
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X,y)

RFE(estimator=LogisticRegression(max_iter=1700, penalty='none'),
    n_features_to_select=20)

In [497]:
var_selected = pd.DataFrame(sorted(zip(map(lambda x: round(x), rfe.ranking_), X.columns)),
                           columns = ['ranking','variable'])
print(var_selected)

    ranking         variable
0         1                K
1         1           K_by_r
2         1         K_by_tau
3         1        K_times_r
4         1      K_times_tau
5         1                S
6         1           S_by_K
7         1           S_by_r
8         1         S_by_tau
9         1        S_times_K
10        1        S_times_r
11        1      S_times_tau
12        1            Value
13        1       Value_by_r
14        1     Value_by_tau
15        1    Value_times_K
16        1    Value_times_S
17        1  Value_times_tau
18        1              tau
19        1         tau_by_r
20        2    Value_times_r
21        3      tau_times_r
22        4                r
23        5       Value_by_S
24        6       Value_by_K


In [498]:
rfe_selected = train_final[var_selected.head(10)['variable'].values]

## Cross Validation

In [451]:
train_final = new.copy()

In [452]:
train_final.iloc[:,:5]

,Value,S,K,tau,r
0,21.670404,431.623898,420,0.341270,0.03013
1,0.125000,427.015526,465,0.166667,0.03126
2,20.691244,427.762336,415,0.265873,0.03116
3,1.035002,451.711658,460,0.063492,0.02972
4,39.553020,446.718974,410,0.166667,0.02962
...,...,...,...,...,...
1675,1.470000,444.397163,475,0.273810,0.03034
1676,34.927317,438.453825,410,0.301587,0.02972
1677,41.127997,428.042219,390,0.170635,0.03188
1678,0.440003,439.081203,480,0.293651,0.02962


In [453]:
X = train_final.values
y = train['BS'].values

In [454]:
from sklearn.model_selection import StratifiedKFold 

kfolds = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

In [499]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_model = LogisticRegression(penalty='none', max_iter = 10000)
error_model_1_cv = cross_val_score(logistic_model,X,y, cv=kfolds, scoring = 'accuracy')
error_model_2_cv = cross_val_score(logistic_model,train_final.iloc[:,:5].values, y, cv=kfolds,scoring='accuracy')
error_model_3_cv = cross_val_score(logistic_model,rfe_selected, y, cv=kfolds,scoring='accuracy')
print("Logisgic Regression: \n")
print("accuracies of 10-folds:",error_model_1_cv,"(mean classification error:",1-np.mean(error_model_1_cv),")")
print("accuracies of 10-folds:",error_model_2_cv,"(mean classification error:",1-np.mean(error_model_2_cv),")")
print("accuracies of 10-folds:",error_model_3_cv,"(mean classification error:",1-np.mean(error_model_3_cv),")")

Logisgic Regression: 

accuracies of 10-folds: [0.95238095 0.95833333 0.92261905 0.9702381  0.96428571 0.91071429
 0.95833333 0.93452381 0.9047619  0.95238095] (mean classification error: 0.05714285714285716 )
accuracies of 10-folds: [0.92261905 0.94642857 0.89285714 0.94642857 0.92857143 0.91071429
 0.94047619 0.9047619  0.88690476 0.93452381] (mean classification error: 0.07857142857142851 )
accuracies of 10-folds: [0.91071429 0.93452381 0.9047619  0.93452381 0.92261905 0.9047619
 0.93452381 0.92857143 0.88095238 0.91071429] (mean classification error: 0.08333333333333326 )


In [500]:
#The model with all the features seems to perform the best with mean class error at .057

## Modeling with all variables

In [501]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [502]:
X = train_final.values
y = train['BS'].values

In [503]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state = 1)

In [504]:
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

# Logistic Regression

In [505]:
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=1700).fit(X_train, y_train)

In [506]:
print(f'Classification Error: {1-mlr.score(X_test, y_test)}')

Classification Error: 0.05654761904761907


In [507]:
#not sure why im getting the warning for the normalized x
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=2000).fit(X_train_norm, y_train)

/Users/anushkakapur/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [508]:
print(f'Classification Error: {1-mlr.score(X_test_norm, y_test)}')

Classification Error: 0.044642857142857095


# LDA

In [509]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train_norm, y_train)

LinearDiscriminantAnalysis()

In [510]:
print(f'Classification Error: {1-lda.score(X_test_norm, y_test)}')

Classification Error: 0.05654761904761907


# KNN

### 5 neighbors

In [511]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_norm, y_train)

KNeighborsClassifier()

In [512]:
print(f'Classification Error: {1-knn.score(X_test_norm, y_test)}')

Classification Error: 0.07738095238095233


### 10 neighbors

In [513]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_norm, y_train)

KNeighborsClassifier(n_neighbors=10)

In [514]:
print(f'Classification Error: {1-knn.score(X_test_norm, y_test)}')

Classification Error: 0.0714285714285714


# Boosted Trees

In [515]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0,
                                 max_depth=1, random_state=1).fit(X_train_norm, y_train)

print(f'Classification Error: {1-clf.score(X_test_norm, y_test)}')

Classification Error: 0.059523809523809534


In [516]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=1,
                                 max_depth=1, random_state=1).fit(X_train_norm, y_train)

print(f'Classification Error: {1-clf.score(X_test_norm, y_test)}')

Classification Error: 0.0535714285714286


# Random Forest

In [517]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train_norm,y_train)
print(f'Classification Error: {1-clf.score(X_test_norm, y_test)}')

Classification Error: 0.0625


In [518]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=200)
clf.fit(X_train_norm,y_train)
print(f'Classification Error: {1-clf.score(X_test_norm, y_test)}')

Classification Error: 0.0625
